### Traffic Flow analysis and forecasting using Machine Learning and Deep Learning Techniques

The data used for this research comes from a project called the cabspotting project. Specifically, this dataset contains about 11,000,000 GPS data of various taxis (Yellow Cab Vehicle) in the San Francisco, California area. The whole data sampling occurred in May 2008.

<b>You can read more about the dataset in the following link: </b>https://stamen.com/work/cabspotting/

### This notebook process the initial dataset, in order to extract usefull information from it. 
The rest of this notebook contains the code that we developed, in order to preprocess the dataset and con

In [1]:
# measure execution time
%load_ext autotime

time: 0 ns (started: 2023-08-02 17:24:55 +03:00)


In [2]:
# import basic libraries
import os
import json
import random
import requests

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

%matplotlib inline

time: 11.3 s (started: 2023-08-02 17:24:55 +03:00)


### Phase 1: Preprocess the dataset
In this step, the following commands are executed:
- Add Taxi ID on the data
- Gather all the data in one txt file
- Convert time information to timestamp
- Split trajectories, based in the time field
- Delete unnecessary columns
- Map Matching using Valhalla Meili API

#### Step 1: Preprocess the initial dataset with raw data

In [ ]:
# define the path in which data are stored
path = 'C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Data'

counter = 0 # Taxi ID starts from 0

# create an empty dataframe, in which all the data will be saved
all_data = pd.DataFrame(columns=['Taxi ID','Latitude','Longitude','Occupied','Date Time'])

for filename in os.listdir(path):
    
    # read each file in the Data folder
    temp = pd.read_csv(path+'/'+filename,names=['Latitude','Longitude','Occupied','Date Time'],sep=' ')
    
    # assign Taxi ID number to each file
    temp.insert(1,'Taxi ID',counter)
    
    # add the data in this file in the 'all_data' dataframe
    all_data = pd.concat([all_data, temp],ignore_index = True)
    
    counter += 1 # Increase Taxi ID number by 1    

##### Change datetime field to timestamp

In [ ]:
all_data['Date Time'] = pd.to_datetime(all_data['Date Time'],origin='unix',unit='s')

In [ ]:
# choose the data of one week
all_data = all_data[(all_data['Date Time'] >= "2008-05-18 00:00:00") & (all_data['Date Time'] < "2008-05-25 00:00:00")]

##### Sort the data based in Taxi ID and timestamp information

In [ ]:
all_data = all_data.sort_values(['Taxi ID','Date Time'])
all_data = all_data.reset_index(drop=True)

##### Delete the 'Occupied' column
This column denotes whether or not the taxi was occupied by a passenger, at the time of GPS recording. So, this information is not usefull for our research.

In [ ]:
all_data.drop('Occupied',axis=1,inplace=True)

#### Step 2: Split the trajectories based in time field and FIle ID

Split each trajectory in the same Taxi ID based in the timestamp field.

Here, <b>n_sec</b> variable denotes the maximum number of seconds that consecutive GPS traces in the same trajectory should have.

In [ ]:
all_data.insert(1,'Traj ID',-1)

In [ ]:
''' 
Each Taxi ID contains GPS data of one trajectory

If the time gap between two GPS points is lower than n_sec seconds, (condition 1)
and these GPS points belong to the same Taxi ID  (condition 2)
then asign the same Traj ID number. (result)

If the time gap between two GPS points is higher than n_sec seconds, (condition 1)
and these GPS points belong to the same Taxi ID  (condition 2)
then asign different Traj ID number to each of these GPS points. (result)

If the GPS points belong to the same Taxi ID  (condition 2)
then asign different Traj ID number to each of these GPS points. (result)

'''

# max number of seconds between GPS records of each traectory
n_sec = 90
traj_id = 0

for i in range(all_data.shape[0] -1):
    
    if (all_data['Taxi ID'][i+1] == all_data['Taxi ID'][i]): # belong to the same File ID
        
        if (((all_data['Date Time'][i+1])-(all_data['Date Time'][i])).total_seconds() <= n_sec): # time interval less-equal than n_sec
            all_data.at[i,'Traj ID'] = traj_id
            all_data.at[i+1,'Traj ID'] = traj_id
            
        else: # time interval higher than n_sec
            all_data.at[i,'Traj ID'] = traj_id
            traj_id +=1
            all_data.at[i+1,'Traj ID'] = traj_id
    
    else: # not belong to the same File ID
        all_data.at[i,'Traj ID'] = traj_id
        traj_id  = 0
        all_data.at[i+1,'Traj ID'] = traj_id

##### Delete trajectories, which contain only one OSM Way ID

In [ ]:
uniques = all_data.loc[:, ['Taxi ID', 'Traj ID']].drop_duplicates(keep=False).index
all_data.drop(uniques,axis=0,inplace=True)
all_data.reset_index(drop=True,inplace=True)

##### Find min and max date in this dataset

In [ ]:
print("Min date is: "+str(all_data['Date Time'].min()))
print("Max date is: "+str(all_data['Date Time'].max()))

#### Step 3: Begin Map Matching

Map matching done using Valhalla Meili API. Given each trajectory to the API as input, the response contains information of the exact path that each trajectory followed. The paths are in the form of OSM Way IDs. 

Sources:

-  <b>Installation using Docker: </b>https://ikespand.github.io/posts/meili/
-  <b>Paper about Valhalla: </b>https://link.springer.com/article/10.1007/s42979-022-01340-5#Tab5
-  <b>APIs documentation: </b>https://valhalla.github.io/valhalla/api/map-matching/api-reference/#matched-point-items  

In [ ]:
# pass lat and lot pairs to Valhalla API
df_for_meili = all_data[['Latitude','Longitude']]
df_for_meili = df_for_meili.rename(columns={"Latitude": "lat", "Longitude": "lon"})

Create a new dataframe under the name "visited_segments", in which information about each trajectory will be contained. The columns of this new dataframe are:
-  <b>File ID: </b>The folder that contains information of this trajecotry.
-  <b>Traj ID: </b>The ID of the trajectory in this folder.
-  <b>OSM Way ID: </b>The way ID number of the edge that trajectory visited.
-  <b>Start Time: </b>Expected time that trajecotry enter the specific edge.
-  <b>End Time: </b>Expected time that trajecotry left the specific edge.

In [ ]:
visited_segments = pd.DataFrame(columns=['Taxi ID','Traj ID','OSM Way ID','Start Time','End Time'])

for taxi_id in all_data['Taxi ID'].unique():
    for traj_id in all_data[all_data['Taxi ID'] == taxi_id]['Traj ID'].unique():

            # get the batch of data that we send to the request
            indexes = all_data[(all_data['Taxi ID']==taxi_id) & (all_data['Traj ID'] == traj_id)].index
            
            # input to API
            passed_data = df_for_meili.iloc[indexes]

            # Preparing the request to Valhalla's Meili
            meili_coordinates = passed_data.to_json(orient='records')
            meili_head = '{"shape":'
            meili_tail = ""","search_radius": 250, "sigma_z": 10, "beta": 10,"shape_match":"map_snap", "costing":"auto",
                            "filters":{"attributes":["edge.way_id"],"action":"include"},
                            "format":"osrm"}"""

            # this is the request
            meili_request_body = meili_head + meili_coordinates + meili_tail

            # the URL of the local valhalla server
            url = "http://localhost:8002/trace_attributes"

            # providing headers to the request
            headers = {'Content-type': 'application/json'}

            # we need to send the JSON as a string
            data = str(meili_request_body)

            # sending a request
            r = requests.post(url, data=data, headers=headers)

            if r.status_code == 200: # response from Valhalla API was successful

                # Parsing the JSON response
                response_text = json.loads(r.text)

                # find the time interval (in sec) that the trajectory needs to be completed [last timestamp - first timestamp]
                interval = (all_data.iloc[indexes].iloc[-1]['Date Time'] - all_data.iloc[indexes].iloc[0]['Date Time']).total_seconds()

                # compute the expected duration that the moving object is in each edge (duration is equal for each edge that the trajectory visits)
                duration  = interval/len(response_text['edges'])

                # make a temporary dataframe
                temp = pd.DataFrame(columns=['Taxi ID','Traj ID','OSM Way ID','Start Time','End Time'])

                # make the final dataframe with the help of a temporary dataframe
                for i in range(len(response_text['edges'])):

                    # complete the fields of temp dataframe
                    temp.at[i,'Taxi ID'] = taxi_id
                    temp.at[i,'Traj ID'] = traj_id
                    temp.at[i,'OSM Way ID'] = response_text['edges'][i]['way_id']

                    if i == 0:
                        temp.at[i,'Start Time'] = all_data.iloc[indexes].iloc[0]['Date Time']
                    else:
                        temp.at[i,'Start Time'] = temp.at[i-1,'End Time']

                    temp.at[i,'End Time'] = temp.at[i,'Start Time'] + timedelta(seconds=duration)

                # concatenate the two dataframes
                visited_segments = pd.concat([visited_segments,temp],ignore_index=True)

##### Delete trajectories, which contain only one OSM Way ID

In [ ]:
uniques = visited_segments.loc[:, ['Taxi ID', 'Traj ID']].drop_duplicates(keep=False).index
visited_segments.drop(uniques,axis=0,inplace=True)
visited_segments.reset_index(drop=True,inplace=True)
visited_segments = visited_segments.sort_values(['Taxi ID','Traj ID','Start Time']).reset_index(drop=True)

In [ ]:
# save the new dataframe to separate txt file
# visited_segments.to_csv('C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Files/visited_segments.txt',index=False)

### Phase 2: Make two time series datasets
- The first time series dataset will contain the traffic flow in each path using the SPQ function
- The second time series dataset will contain the traffic flow in each path without the use of the SPQ function

In [3]:
# read and sort the data. Also, convert timestamps to datetime data type
visited_segments = pd.read_csv('C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Files/visited_segments.txt')
visited_segments['Start Time'] = pd.to_datetime(visited_segments['Start Time'],format='%Y-%m-%d %H:%M:%S.%f')
visited_segments['End Time'] = pd.to_datetime(visited_segments['End Time'],format='%Y-%m-%d %H:%M:%S.%f')

time: 1min 54s (started: 2023-08-02 17:25:15 +03:00)


#### Step 1: Create the time information of the time series datasets

In [4]:
# find max and min timestamp in the dataset
min_timestamp = visited_segments['Start Time'].min()
print("Min timestamp value in the dataframe is: ",min_timestamp)

max_timestamp = visited_segments['End Time'].max()
print("Max timestamp value in the dataframe is: ",max_timestamp)


# calculate total seconds between those max and min values
total_sec = (max_timestamp-min_timestamp).total_seconds()
print("\nTotal duration in sec in this dataframe is: ",total_sec)

Min timestamp value in the dataframe is:  2008-05-18 00:00:00
Max timestamp value in the dataframe is:  2008-05-24 23:59:59.000130

Total duration in sec in this dataframe is:  604799.00013
time: 218 ms (started: 2023-08-02 17:27:10 +03:00)


Since we have data of one week, we will create time intervals  of half hour

In [5]:
# this list contains the time information of our time-series data
time_info = []

i =0
while(True):
    if i == 0:
        time_info.append(min_timestamp)
    else:
        time_info.append(time_info[i-1] + timedelta(seconds=1800))
    
    if (time_info[i]>=max_timestamp):
        break
    
    i+=1 

# create pairs of consequtive values of the list time_info
time_intervals = list(zip(*[time_info[i:] for i in range(2)])) 

time: 31 ms (started: 2023-08-02 17:27:10 +03:00)


#### Step 2: Generate random unique paths of random lengths
These paths can be of any length. The number of consecutive edges contained in the path define it's length. The code below uses the information created above (visited_segments), in order to generate the paths.

In [6]:
paths = pd.DataFrame(columns=['Taxi ID','Traj ID','Path','Length']) # empty dataframe, in which all the paths will be saved

length_options = [2,3,4,5,6,7,8,9,10,11,12,13,14,15] # define the length options

check = [] # this list is used, in order to be sure that paths created are unique

indexes = np.arange(visited_segments.shape[0]-max(length_options)+1) # legal indexes in which algorithm can search for paths

i = 0
n = 1000 # number of paths to be created

while (paths.shape[0] < n):
    p = [] # this list will save the path created
    selected_index = random.choice(indexes) # select randomly an index   
    selected_length = random.choice(length_options) # the length of the new path to be created

    # check if path is within the same trajectory
    if ((visited_segments.at[selected_index,'Taxi ID'] == visited_segments.at[selected_index+(selected_length-1),'Taxi ID']) & 
        (visited_segments.at[selected_index,'Traj ID'] == visited_segments.at[selected_index+(selected_length-1),'Traj ID'])):
        
        # iterate to the visited_segments dataframe and extract the consequtive 
        # [selected_index...index+selected_length] OSM Way IDs
        for y in range(selected_length): 
            p.append(int(visited_segments.at[selected_index+y,'OSM Way ID']))
    
        # if path is unique, then add it to paths dataframe
        if p not in check: 
            check.append(p)
            paths.at[i,'Taxi ID'] = visited_segments.at[selected_index,'Taxi ID']
            paths.at[i,'Traj ID'] = visited_segments.at[selected_index,'Traj ID']
            paths.at[i,'Path'] = p # add the new path to the dataframe
            paths.at[i,'Length'] = selected_length
            i += 1
        else:
            continue      

time: 1 s (started: 2023-08-02 17:27:10 +03:00)


#### Step 3a: Create the SPQ function

This is the main function that will be used for the construction of the time series dataset.
The SPQ (Strict Path Query) function, returns all the trajectories [the trajectories are unique (Taxi_ID,Traj_ID) pairs] that passes through given path of edges at a given time interval [time_enter,time_leave].

Parameters:
- <b>path: </b> The path that the tajectories should EXACTLY follow (edge by edge). This path can be of any length greater or equal to 2 edges.

- <b>time_enter: </b>The time, in which the trajectory should enter the first edge of the path given as input.
- <b>time_leave: </b>The time, in which the trajectory should leave the last edge of the path given as input.

In [7]:
def SPQ(path,time_enter,time_leave):
    
    # length of the path given to the function
    path_length = len(path)
    
    # this list will save temporarily the trajectories that match the SPQ condition
    trajectories = []
    
    # extract only the data that match the time interval given as input
    #examined_data = visited_segments[(visited_segments['Start Time'] >= time_enter) & (visited_segments['End Time'] <= time_leave)].reset_index(drop=True)

    # find all the indexes, in which the first edge in the path is located
    needed_indexes = examined_data[examined_data['OSM Way ID'] == path[0]].index

    # iterate through all indexes (note the Taxi_ID and Traj_ID numbers)
    for index in needed_indexes:

        traj_id = examined_data.at[index,'Traj ID']
        taxi_id = examined_data.at[index,'Taxi ID']
        inter = 1
        
        # decide if the row in the next index matches the criteria (same Taxi_ID, same Traj_ID, the path required)
        for i in range(1,path_length):
            try:
                if (not ((examined_data['OSM Way ID'].iloc[index+i] == path[i]) 
                         & (examined_data['Traj ID'].iloc[index+i] == traj_id) 
                        & (examined_data['Taxi ID'].iloc[index+i] == taxi_id))):

                    break

                inter += 1 # if the criteria matches, then increase inter counter by one

            except:
                pass # index out of bounds exception (do nothing)
            
        # if the criteria matches as many times as the length of the path, then we found one trajectory
        if (path_length == inter):
            trajectories.append((taxi_id,traj_id)) # add this trajectory to the trajectories list
    
    # return the number of trajectories that matches the criteria
    return (len([t for t in (set(tuple(i) for i in trajectories))]))

time: 0 ns (started: 2023-08-02 17:27:11 +03:00)


#### Step 3b: Create the same SPQ function using postgreSQL

In order to do this, we have loaded the visited_segments data into a table called "visited_segments". The SPQ function has been rewritten using PL/pgSQL language. In order to accelerate the execution of this function, we have defined B+ tree indexes on columns of the visited_segments table. The function is located inside the database.

#### Compare the two methods, using plots. The comparation is based in CPU time.
In order to be more acurate, we are executing batches of 20 queries (with paths of same length) using python and PL/pgSQL implementation of SPQ function. The execution of PL/pgSQL function is done inside the pgAdmin4, in order not to include the request and responce times.

Comments:

1) The vertical axis shows the execution time in seconds. As for postgreSQL, the queries are executed inside pgAdmin4, so we can measure the net runtime.

2) The horizontal axis includes the batches of queries executed at a time. Essentially, we execute 20 queries at a time with paths of equal length. The exact same queries are executed in python and in postgreSQL. The horizontal axis essentially reports the length of the paths contained each time in the batches of 20 queries executed.

3) Each bar essentially reports the total execution time of 20 SPQ calls with path length path length.

4) Faster implementation in python

In [ ]:
from IPython import display
display.Image("C:/Users/SK/Desktop/Diploma_Thesis/SPQ_python_plpgsql.png", alt="SPQ calls of batches of 20 queries in pythons vs in plpgSQL",height=400,width=600)

#### Step 4a: Fill the time series dataframe using the SPQ function
This dataset to be created will describe at each path and at each time interval the traffic flow at each path using the rules described in the following paper:

-  <b>SQP query paper: </b>https://dl.acm.org/doi/abs/10.1145/2666310.2666413

In [8]:
# create an empty dataframe
time_series_SPQ = pd.DataFrame(columns=time_intervals)

# fill the time series dataset
time_series_SPQ.insert(0,'Taxi ID',0)
time_series_SPQ.insert(1,'Traj ID',0)
time_series_SPQ.insert(2,'Path',0)
time_series_SPQ.insert(3,'Length',0)
time_series_SPQ['Path'] = paths['Path']
time_series_SPQ['Length'] = paths['Length']
time_series_SPQ['Taxi ID'] = paths['Taxi ID']
time_series_SPQ['Traj ID'] = paths['Traj ID']

time: 78 ms (started: 2023-06-23 19:27:42 +03:00)


In [9]:
# fill the dataframe column by column
for time in time_intervals:
    i = 0
    examined_data = visited_segments[(visited_segments['Start Time'] >= time[0]) & (visited_segments['End Time'] <= time[1])].reset_index(drop=True)
    for path in paths['Path'].to_list():
        time_series_SPQ.at[i,time] = SPQ(path,time[0],time[1])
        i += 1

time: 1h 2min 41s (started: 2023-06-23 19:27:42 +03:00)


In [13]:
# print the dataframe with SPQ traffic flow information
time_series_SPQ

,Taxi ID,Traj ID,Path,Length,"(2008-05-18 00:00:00, 2008-05-18 00:05:00)","(2008-05-18 00:05:00, 2008-05-18 00:10:00)","(2008-05-18 00:10:00, 2008-05-18 00:15:00)","(2008-05-18 00:15:00, 2008-05-18 00:20:00)","(2008-05-18 00:20:00, 2008-05-18 00:25:00)","(2008-05-18 00:25:00, 2008-05-18 00:30:00)",...,"(2008-05-24 23:10:00, 2008-05-24 23:15:00)","(2008-05-24 23:15:00, 2008-05-24 23:20:00)","(2008-05-24 23:20:00, 2008-05-24 23:25:00)","(2008-05-24 23:25:00, 2008-05-24 23:30:00)","(2008-05-24 23:30:00, 2008-05-24 23:35:00)","(2008-05-24 23:35:00, 2008-05-24 23:40:00)","(2008-05-24 23:40:00, 2008-05-24 23:45:00)","(2008-05-24 23:45:00, 2008-05-24 23:50:00)","(2008-05-24 23:50:00, 2008-05-24 23:55:00)","(2008-05-24 23:55:00, 2008-05-25 00:00:00)"
0,170,558,"[512523366, 512523366, 512523366]",3,1,2,1,1,0,0,...,2,2,1,2,2,1,1,1,0,0
1,28,308,"[225847523, 225847523, 225847523, 225847523, 2...",8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,429,101,"[417400703, 247966218, 247966218, 247966218, 2...",12,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,320,77,"[224384015, 224384015, 224384015, 224384015, 2...",7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,335,117,"[706532925, 417400734, 417400734, 706532923, 7...",11,0,0,1,1,0,1,...,0,0,1,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,393,11,"[306338108, 306338108]",2,1,0,1,1,0,1,...,3,7,6,4,1,2,1,2,2,1
996,241,23,"[91181759, 100400946, 100400946, 674757876, 67...",8,1,0,1,0,1,0,...,0,0,0,1,0,1,1,3,0,1
997,23,71,"[162799061, 162799061, 162799061, 162799061, 1...",7,0,0,0,2,0,0,...,0,0,0,0,0,0,1,2,0,0
998,206,122,"[417389565, 417389569, 417389569, 417389547, 5...",8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


time: 62 ms (started: 2023-06-23 20:49:19 +03:00)


In [11]:
time_series_SPQ.to_csv('C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Files/time_series_SPQ.txt',index=False)

time: 891 ms (started: 2023-06-23 20:30:23 +03:00)


#### Step 4b: Fill the time series dataframe without using the SPQ function
In this case, the traffic flow information at each path and at each time interval does not follow the conditions and rules that the SPQ function denotes. So in this dataset, traffic flow at each path will describe that in a specific time interval, a number of moving objects have crossed all the edge ids (roads) contained in a specific path.

In [ ]:
# create an empty dataframe
time_series_without_SPQ = pd.DataFrame(columns=time_intervals)

# fill the time series dataset
time_series_without_SPQ.insert(0,'Taxi ID',0)
time_series_without_SPQ.insert(1,'Traj ID',0)
time_series_without_SPQ.insert(2,'Path',0)
time_series_without_SPQ.insert(3,'Length',0)
time_series_without_SPQ['Path'] = paths['Path']
time_series_without_SPQ['Length'] = paths['Length']
time_series_without_SPQ['Taxi ID'] = paths['Taxi ID']
time_series_without_SPQ['Traj ID'] = paths['Traj ID']

##### Use a function to fill the times series dataset without the use of SPQ
The given code defines a function called traffic_flow_without_SPQ that takes a list <b>path</b> and two time parameters <b>time_enter</b> and <b>time_leave</b>. It aims to find the count of trajectories in some data, where the given path is entirely covered by the 'OSM Way ID' values associated with each trajectory and taxi, based on the specified time interval.

In [ ]:
def traffic_flow_without_SPQ(path, time_enter, time_leave):
    # Initialize the count of matching trajectories
    trajectories = 0
    
    # extract only the data that match the time interval given as input
    # examined_data = visited_segments[(visited_segments['Start Time'] >= time_enter) & (visited_segments['End Time'] <= time_leave)].reset_index(drop=True)

    # find the indexes of entries in 'examined_data' where 
    # the first element of the 'path' is present in the 'OSM Way ID' column
    needed_indexes = examined_data[examined_data['OSM Way ID'] == path[0]].index

    # convert the 'path' list to a set for efficient subset check
    path_roads = set(path)

    # loop through the needed indexes to check for matching trajectories
    for index in needed_indexes:
        
        # get the 'Traj ID' and 'Taxi ID' for the current index
        traj_id = examined_data.at[index, 'Traj ID']
        taxi_id = examined_data.at[index, 'Taxi ID']

        # find all the 'OSM Way ID' values for the current 'Traj ID' and 'Taxi ID'
        selected_roads = set(examined_data[(examined_data['Traj ID'] == traj_id) & (examined_data['Taxi ID'] == taxi_id)]['OSM Way ID'])
        
        # check if the 'path_roads' is a subset of 'selected_roads'
        if path_roads.issubset(selected_roads):
            
            # increment the count of matching trajectories
            trajectories += 1

    # return the count of matching trajectories
    return trajectories


In [ ]:
# fill the dataframe column by column
for time in time_intervals:
    i = 0
    examined_data = visited_segments[(visited_segments['Start Time'] >= time[0]) & (visited_segments['End Time'] <= time[1])].reset_index(drop=True)
    for path in paths['Path'].to_list():
        time_series_without_SPQ.at[i,time] = traffic_flow_without_SPQ(path,time[0],time[1])
        i += 1

In [ ]:
# print the dataframe without the SPQ traffic flow information
time_series_without_SPQ

In [ ]:
# save dataframe to CSV format
time_series_without_SPQ.to_csv('C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Files/time_series_without_SPQ.txt',index=False)

##### Plot the distribution of paths created with respect to the length

In [1]:
# count the number of paths grouped by length attribute
plt.figure(figsize=(12,5))
sns.countplot(time_series,x='Length')
plt.xlabel('Length inputs')
plt.ylabel('Number of Paths with specific Length')
plt.title('Distribution of path lenghts in the two datasets')
plt.show()

NameError: name 'plt' is not defined